In [64]:
%reload_ext autoreload
%autoreload 2

In [2]:
import h5py
data_path = 'data/clip768v2/10M/dataset.h5'
%time f = h5py.File(data_path, 'r')

CPU times: user 0 ns, sys: 1.08 s, total: 1.08 s
Wall time: 1.18 s


In [3]:
len(f['emb'])

10120191

## Goal:
1) load data for training (1000*1000 objects)
2) train, predict on train, dispose of the training data (keep ids)
3) load data for predict (rest 9M objects)
4) predict, dispose of the data (keep ids)
5) on search, load the data in the visited bucket again (10k objects on avg), compute distances

In [3]:
%time loaded_data = f['emb'][:, :]

CPU times: user 35.5 ms, sys: 16.3 s, total: 16.4 s
Wall time: 2min 43s


In [4]:
from li.model import Model
model = Model(input_dim=768, output_dim=100)

In [9]:
import torch
model.load_state_dict(torch.load('../models/clip768v2-emb-10M-best-model.pt'))
model.eval()

Model(
  (layers): Sequential(
    (0): Linear(in_features=768, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=100, bias=True)
  )
)

In [13]:
from li.model import data_X_to_torch, predict, get_device

In [14]:
%time predictions = predict(model, get_device(), data_X_to_torch(loaded_data))

CPU times: user 1min 59s, sys: 47.8 s, total: 2min 47s
Wall time: 2min 51s


In [15]:
predictions.shape

(10120191,)

In [47]:
from li.model import predict_proba

In [27]:
%time f2 = h5py.File('data/clip768v2/10M/query.h5', 'r')
[k for k in f2.keys()]

CPU times: user 460 µs, sys: 250 µs, total: 710 µs
Wall time: 16.1 ms


['emb']

In [28]:
%time loaded_queries = f2['emb'][:, :]

CPU times: user 1.36 ms, sys: 8.74 ms, total: 10.1 ms
Wall time: 10.2 ms


In [33]:
loaded_queries.shape

(10000, 768)

In [48]:
res = predict_proba(model, get_device(), data_X_to_torch(loaded_queries))

torch.Size([10000, 100])


In [66]:
from li.LearnedIndex10M import LearnedIndex

In [67]:
li = LearnedIndex(
    dataset_shape=(10_000_000, 768),
    n_categories=100
)

In [68]:
li.model = model

In [69]:
li.predictions = predictions

In [70]:
li.all_data =  loaded_data

In [74]:
%%time
search_res = li.search(loaded_queries, n_buckets=2)

bucket_obj_indexes: (70660,)
bucket_obj_indexes: (60431,)
bucket_obj_indexes: (155211,)
bucket_obj_indexes: (44668,)
bucket_obj_indexes: (44940,)
bucket_obj_indexes: (35358,)
bucket_obj_indexes: (78389,)
bucket_obj_indexes: (46592,)
bucket_obj_indexes: (120995,)
bucket_obj_indexes: (43151,)
bucket_obj_indexes: (145368,)
bucket_obj_indexes: (65688,)
bucket_obj_indexes: (42585,)
bucket_obj_indexes: (151551,)
bucket_obj_indexes: (142066,)
bucket_obj_indexes: (297296,)
bucket_obj_indexes: (109030,)
bucket_obj_indexes: (76712,)
bucket_obj_indexes: (85640,)
bucket_obj_indexes: (138392,)
bucket_obj_indexes: (97728,)
bucket_obj_indexes: (102276,)
bucket_obj_indexes: (150129,)
bucket_obj_indexes: (130399,)
bucket_obj_indexes: (140012,)
bucket_obj_indexes: (34989,)
bucket_obj_indexes: (136013,)
bucket_obj_indexes: (117123,)
bucket_obj_indexes: (89756,)
bucket_obj_indexes: (36506,)
bucket_obj_indexes: (108078,)
CPU times: user 5min 24s, sys: 27 s, total: 5min 51s
Wall time: 6min


In [76]:
search_res.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [81]:
search_res[0].shape

(10000, 10)

In [80]:
search_res[1]

array([[ 1831320,  7535354,  7059014, ...,  9060635,  5509896,   210839],
       [  345243,   641543,   985275, ...,  4315057,  6363149,  4984877],
       [ 1385230,  8732144,  5079078, ...,  8500628,  7238113,  8391052],
       ...,
       [ 9550551,  9711325,  7029902, ...,  7054237,  8342966,  2623128],
       [10037932,  6595916,  4837121, ...,  2597817,  8233243,  3702635],
       [  497148,  8333824,  4361805, ...,  7069212,  7326701,  5715343]],
      dtype=uint32)

In [79]:
res[1].shape

(100,)

In [44]:
import numpy as np

In [46]:
res[0]

array([8.37511470e-05, 9.70561174e-04, 2.92020995e-04, 1.63676043e-04,
       8.52210354e-03, 7.81660492e-05, 1.58739305e-04, 6.29106071e-04,
       1.61633524e-03, 1.29780790e-04, 2.15484848e-04, 2.32058756e-05,
       1.46150356e-04, 2.36730743e-03, 1.38364936e-04, 1.30514824e-03,
       3.36273294e-03, 6.12222531e-04, 7.04983031e-05, 2.39886431e-04,
       1.60547465e-01, 1.37590361e-03, 2.52572994e-04, 1.07597589e-04,
       1.17151719e-03, 1.18067735e-04, 6.09984563e-04, 2.42916425e-03,
       3.16633657e-03, 4.98839188e-03, 1.29831792e-03, 8.74745543e-04,
       6.43437961e-04, 2.13510357e-03, 3.41637860e-05, 4.37771529e-03,
       2.57312728e-04, 7.93551953e-05, 2.43538052e-05, 1.43318884e-02,
       3.92020098e-04, 5.58090760e-05, 1.26074348e-03, 3.68411615e-02,
       4.93923551e-04, 1.02681312e-04, 1.21077430e-03, 2.71683879e-04,
       1.73655041e-02, 2.19049998e-05, 4.27135747e-05, 3.40649131e-05,
       1.30197138e-03, 2.38854840e-01, 3.57980825e-05, 4.40446602e-05,
      

In [45]:
np.argmax(res[0])

80

In [37]:
res[0].shape, res[1].shape

((10, 100), (10, 100))

In [16]:
predictions[:10]

array([38,  3, 58, 88, 84, 67, 13, 93, 75, 13])

In [ ]:
torch.save(model.state_dict(), f'{save_dir}/{wandb.run.name}/best-model.pt')


In [7]:
import numpy as np

In [9]:
def get_random_indexes(size, dataset_size, seed):
    rng = np.random.default_rng(seed)
    random_idxs = rng.choice(range(dataset_size), size=(size), replace=False)
    random_idxs = np.sort(random_idxs)
    return random_idxs

idx = get_random_indexes(1000, 10_000_000, 2023)
idx.shape

(1000,)

In [10]:
idx.shape == (1000, )

True

In [7]:
%time loaded_data = f['emb'][random_idxs, :]

CPU times: user 2.17 s, sys: 784 ms, total: 2.95 s
Wall time: 1min 19s


In [3]:
%time _ = f['emb'][:100_000, :]

CPU times: user 13.7 ms, sys: 110 ms, total: 124 ms
Wall time: 2.05 s


In [4]:
%time _ = f['emb'][:1_000_000, :]

CPU times: user 12.7 ms, sys: 1.03 s, total: 1.05 s
Wall time: 20.7 s


In [5]:
%time _ = f['emb'][:9_000_000, :]

CPU times: user 11.6 ms, sys: 16.1 s, total: 16.1 s
Wall time: 3min 10s


In [8]:
%time pred_data = f['emb'][:, :]

CPU times: user 211 ms, sys: 14.5 s, total: 14.7 s
Wall time: 4min 4s


In [ ]:
pred_data[]

In [ ]:
%time test_mem = f['emb'][random_idxs, :]

In [11]:
%%time
rng = np.random.default_rng(2023)
print(rng)
random_pivots = rng.choice(range(10_000_000), size=(1_000), replace=False)
pivot_indexes = np.sort(random_pivots)
pivot_data = f['emb'][pivot_indexes, :]

Generator(PCG64)
CPU times: user 1.1 s, sys: 529 ms, total: 1.63 s
Wall time: 10.8 s


In [12]:
pivot_data.shape

(1000, 768)

In [4]:
loaded_data.shape

(10000, 768)

In [16]:
from tqdm import tqdm
import gc
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

def pairwise_cosine(x, y):
    r = cosine_similarity(x, y)
    print(r)
    return 1-cosine_similarity(x, y)

In [28]:
pivot_data[0].unsqueeze()

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [36]:
loaded_data.shape

(1000, 768)

In [52]:
dists_all.shape

(2, 1000)

In [55]:
pivot_data.shape

(1000, 768)

In [59]:
training_data = np.empty((0,768))
training_data

array([], shape=(0, 768), dtype=float64)

In [17]:
%%time
dists_all = np.empty((0,1000))
training_data = np.empty((0,768))
prev = 0

for batch, pivot in zip(range(10_000, 10_000_000, 10_000), pivot_data):
    loaded_data = f['emb'][prev:batch, :]
    dists = np.argsort(cosine_similarity([pivot], loaded_data)[0])
    training_data_indexes = dists[-1000:]
    print(training_data_indexes.shape)
    break
    # take the 1000 most similar (argsort sorts from lowest to highest)
    dists_all = np.vstack((dists_all, prev+training_data_indexes))
    training_data = np.vstack((training_data, loaded_data[training_data_indexes]))
    del loaded_data
    gc.collect()
    prev = batch

(1000,)
CPU times: user 44.2 ms, sys: 20.1 ms, total: 64.3 ms
Wall time: 84 ms


In [75]:
training_data.shape

(999000, 768)

In [13]:
10_000_000 // 1000

10000

In [95]:
%%time
labels = np.array([np.array([i for j in range(1000)]) for i in range(999)])
labels = labels.reshape(labels.shape[0]*labels.shape[1])

CPU times: user 106 ms, sys: 2.27 ms, total: 108 ms
Wall time: 121 ms


In [69]:
dists_all.shape

(10, 1000)

In [87]:
from typing import Tuple
def get_device() -> torch.device:
    """ Gets the `device` to be used by torch.
    This arugment is needed to operate with the PyTorch model instance.

    Returns
    ------
    torch.device
        Device
    """
    use_cuda = torch.cuda.is_available()
    device = torch.device('cuda:0' if use_cuda else 'cpu')
    torch.backends.cudnn.benchmark = True
    return device


def data_X_to_torch(data) -> torch.FloatTensor:
    """ Creates torch training data."""
    data_X = torch.from_numpy(np.array(data).astype(np.float32))
    return data_X


def data_to_torch(data, labels) -> Tuple[torch.FloatTensor, torch.LongTensor]:
    """ Creates torch training data and labels."""
    data_X = data_X_to_torch(data)
    data_y = torch.as_tensor(torch.from_numpy(labels), dtype=torch.long)
    return data_X, data_y

In [113]:
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self, input_dim=768, output_dim=1000):
        super().__init__()
        self.layers = torch.nn.Sequential(
          torch.nn.Linear(input_dim, 128),
          torch.nn.ReLU(),
          torch.nn.Linear(128, output_dim)
        )

    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        outputs = self.layers(x)
        return outputs

model = Model()
lr = 0.01
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
device = get_device()

In [98]:
%%time
data_X, data_y = data_to_torch(training_data, labels)
data_X.shape, data_y.shape

CPU times: user 756 ms, sys: 741 ms, total: 1.5 s
Wall time: 1.6 s


(torch.Size([999000, 768]), torch.Size([999000]))

In [135]:
def train(
    data_X: torch.FloatTensor,
    data_y: torch.LongTensor,
    model,
    optimizer,
    device,
    epochs=500,
):
    step = data_X.shape[0] // epochs // 10
    losses = []
    for ep in range(epochs):
        if ep % step == 0 and ep != 0:
            print(f'{time.time()} | Epoch {ep} | Loss {curr_loss.item()}')
        pred_y = model(data_X.to(device))
        curr_loss = loss(pred_y, data_y.to(device))
        losses.append(curr_loss.item())

        model.zero_grad()
        curr_loss.backward()

        optimizer.step()
    return losses, model


In [111]:
%%time
rng_offset = 1
rng = np.random.default_rng(2023+rng_offset)
rng_offset += 1
print(rng)
random_idxs = rng.choice(range(999000), size=(10_000), replace=False)

Generator(PCG64)
CPU times: user 99 ms, sys: 39.6 ms, total: 139 ms
Wall time: 139 ms


In [112]:
data_X_s = data_X[random_idxs]
data_y_s = data_y[random_idxs]
data_X_s.shape, data_y_s.shape

(torch.Size([10000, 768]), torch.Size([10000]))

In [136]:
model = Model(output_dim=np.unique(data_y_s).shape[0])
lr = 0.1
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
device = get_device()

In [137]:
%time losses, model = train(data_X_s, data_y_s, model, optimizer, device, epochs=100)

Epoch 10
Epoch 20
Epoch 30
Epoch 40
Epoch 50
Epoch 60
Epoch 70
Epoch 80
Epoch 90
CPU times: user 55.6 s, sys: 2.76 s, total: 58.3 s
Wall time: 59.8 s


In [141]:
def predict( model, device, data_X: torch.FloatTensor):
    """ Collects predictions for multiple data points (used in structure building)."""
    model = model.to(device)
    model.eval()

    all_outputs = torch.tensor([], device=device)
    with torch.no_grad():
        outputs = model(data_X.to(device))
        all_outputs = torch.cat((all_outputs, outputs), 0)

    _, y_pred = torch.max(all_outputs, 1)
    return y_pred.cpu().numpy()

In [142]:
predictions = predict(model, device, data_X_s)

In [146]:
data_y_s

tensor([515, 589, 280,  ..., 141, 934,  88])

In [145]:
predictions

array([515, 589, 280, ..., 141, 934,  88])

### we can let go of the training data

In [147]:
del training_data
gc.collect()

67

In [ ]:
# load everyting and then filter out whatever
%time pred_data = f['emb'][:, :]

In [157]:
%%time
predict_indexes = np.setdiff1d(np.arange(0, 10_000_000), dists_all.flatten())
predict_indexes.shape

CPU times: user 306 ms, sys: 367 ms, total: 672 ms
Wall time: 813 ms


(9001000,)

In [158]:
%%time
predict_indexes = np.sort(predict_indexes)
predict_indexes.shape

CPU times: user 102 ms, sys: 67.5 ms, total: 170 ms
Wall time: 172 ms


(9001000,)

In [159]:
predict_indexes[:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
%%time
loaded_pred_data = f['emb'][predict_indexes, :]
loaded_pred_data.shape

In [ ]:
loaded_pred_data[:10]

In [155]:
dists_all.reshape((dists_all[0]*dists_all[1]))#.shape

ValueError: maximum supported dimension for an ndarray is 32, found 1000

In [ ]:
    def predict(self, data_X: torch.FloatTensor):
        """ Collects predictions for multiple data points (used in structure building)."""
        self.model = self.model.to(self.device)
        self.model.eval()

        all_outputs = torch.tensor([], device=self.device)
        with torch.no_grad():
            outputs = self.model(data_X.to(self.device))
            all_outputs = torch.cat((all_outputs, outputs), 0)

        _, y_pred = torch.max(all_outputs, 1)
        return np.array([self.model.output_neurons[label] for label in y_pred.cpu().numpy()])

    def predict_single(self, data_X: torch.FloatTensor):
        """ Collects predictions for a single data point (used in query predictions)."""
        self.model = self.model.to(self.device)
        self.model.eval()

        with torch.no_grad():
            outputs = self.model(data_X.to(self.device))

        prob = nnf.softmax(outputs, dim=0)
        top_prob, top_class = prob.topk(self.model.n_output_neurons, dim=0)
        top_prob = top_prob.cpu().numpy()
        return top_prob, np.array([self.model.output_neurons[label] for label in top_class.cpu().numpy()])
